In this notebook we use publicly available health care data to illustrate the use of some python libraries. We will plot health care expenditure per municipality and how health care varies with age. In the process you will see how libraries like pandas and matplotlib work.




# Health care expenditures per municipality



We start with health care data on the level of municipalities. We will plot how health care expenditure varies with location.




## loading the data



The following code block loads the data. We use data from [Vektis](http://www.vektis.nl/index.php/vektis-open-data). We import the data, which is a 'csv' file with ";" as separator between columns. We also import the pandas library.

Then we inspect the first ten rows of our data set.



In [1]:
# First lets clear all previous python imports and variables by resetting the python kernel.
%reset -f


import pandas as pd

df_gemeente = pd.read_csv('Vektis_Open_Databestand_Zorgverzekeringswet_2014_-_gemeente.csv', sep = ';')

df_gemeente.head(10)

kitten-purple-cardinal-quebec

  GESLACHT LEEFTIJDSKLASSE   GEMEENTENAAM  AANTAL_BSN  AANTAL_VERZEKERDEJAREN  \
0      NaN             NaN            NaN      298383               185664.92
1        M   0 t/m  4 jaar    AA EN HUNZE         507                  468.83
2        M   0 t/m  4 jaar        AALBURG         428                  387.28
3        M   0 t/m  4 jaar       AALSMEER         876                  790.65
4        M   0 t/m  4 jaar         AALTEN         667                  600.00
5        M   0 t/m  4 jaar  ACHTKARSPELEN         810                  737.03
6        M   0 t/m  4 jaar   ALBLASSERDAM         637                  561.21
7        M   0 t/m  4 jaar  ALBRANDSWAARD         735                  663.22
8        M   0 t/m  4 jaar        ALKMAAR        2967                 2660.93
9        M   0 t/m  4 jaar         ALMELO        1906                 1714.63

   KOSTEN_MEDISCH_SPECIALISTISCHE_ZORG  KOSTEN_FARMACIE  \
0                          48661669.94       921

We are going to be interested in total health care costs under the deductible. Hence, we specify the categories that fall under the deductible in the Netherlands in 2014.
Further, we want to do this analysis in English. So we relabel the relevant column names into English.

We specify a list of columns (cost categories) across which we want to add costs. As the costs are total costs (not costs per head) we can indeed add these numbers. We leave out mental health care as it features its own financial incentives.

We specify the name of a new variable `health_expenditure_under_deductible` which is calculated on the list of columns specified and then the function that needs to be applied &#x2013;~sum~ in this case. Finally, we specify that the function needs to be applied row-wise: `axis=1`.

Then we give a dictionary with "old" variable names (in Dutch) and the "new" names in English for the variables where we want to use the English names.



In [1]:
cost_categories_under_deductible = ['KOSTEN_MEDISCH_SPECIALISTISCHE_ZORG', 'KOSTEN_MONDZORG', 'KOSTEN_FARMACIE', 'KOSTEN_HULPMIDDELEN', 'KOSTEN_PARAMEDISCHE_ZORG_FYSIOTHERAPIE', 'KOSTEN_PARAMEDISCHE_ZORG_OVERIG', 'KOSTEN_ZIEKENVERVOER_ZITTEND', 'KOSTEN_ZIEKENVERVOER_LIGGEND', 'KOSTEN_GRENSOVERSCHRIJDENDE_ZORG', 'KOSTEN_GERIATRISCHE_REVALIDATIEZORG', 'KOSTEN_OVERIG']
df_gemeente['health_expenditure_under_deductible'] = df_gemeente[cost_categories_under_deductible].sum(axis=1)

df_gemeente = df_gemeente.rename_axis({
'GESLACHT':'sex',
'LEEFTIJDSKLASSE':'age',
'GEMEENTENAAM':'MUNICIPALITY',
'AANTAL_BSN':'number_citizens',
'KOSTEN_MEDISCH_SPECIALISTISCHE_ZORG':'hospital_care',
'KOSTEN_FARMACIE':'pharmaceuticals',
'KOSTEN_TWEEDELIJNS_GGZ':'mental_care',
'KOSTEN_HUISARTS_INSCHRIJFTARIEF':'GP_capitation',
'KOSTEN_HUISARTS_CONSULT':'GP_fee_for_service',
'KOSTEN_HUISARTS_OVERIG':'GP_other',
'KOSTEN_MONDZORG':'dental care',
'KOSTEN_PARAMEDISCHE_ZORG_FYSIOTHERAPIE':'physiotherapy',
'KOSTEN_KRAAMZORG':'maternity_care',
'KOSTEN_VERLOSKUNDIGE_ZORG':'obstetrics'
}, axis='columns')

df_gemeente.dtypes

charlie-jig-red-texas

sex                                     object
age                                     object
MUNICIPALITY                            object
number_citizens                          int64
AANTAL_VERZEKERDEJAREN                 float64
hospital_care                          float64
pharmaceuticals                        float64
mental_care                            float64
GP_capitation                          float64
GP_fee_for_service                     float64
GP_other                               float64
KOSTEN_HULPMIDDELEN                    float64
dental care                            float64
physiotherapy                          float64
KOSTEN_PARAMEDISCHE_ZORG_OVERIG        float64
KOSTEN_ZIEKENVERVOER_ZITTEND           float64
KOSTEN_ZIEKENVERVOER_LIGGEND           float64
maternity_care                         float64
obstetrics                             float64
KOSTEN_GENERALISTISCHE_BASIS_GGZ       float64
KOSTEN_GRENSOVERSCHRIJDENDE_ZORG     

We drop the columns that we no longer need.



In [1]:
df_gemeente.drop(['AANTAL_VERZEKERDEJAREN',
'KOSTEN_HULPMIDDELEN',
'KOSTEN_PARAMEDISCHE_ZORG_OVERIG',
'KOSTEN_ZIEKENVERVOER_ZITTEND',
'KOSTEN_ZIEKENVERVOER_LIGGEND',
'KOSTEN_GRENSOVERSCHRIJDENDE_ZORG',
'KOSTEN_GERIATRISCHE_REVALIDATIEZORG',
'KOSTEN_OVERIG',
'KOSTEN_GENERALISTISCHE_BASIS_GGZ',
'KOSTEN_EERSTELIJNS_ONDERSTEUNING'],inplace=True,axis=1)
df_gemeente.columns

kitten-quebec-hawaii-william

Index(['sex', 'age', 'MUNICIPALITY', 'number_citizens', 'hospital_care',
       'pharmaceuticals', 'mental_care', 'GP_capitation', 'GP_fee_for_service',
       'GP_other', 'dental care', 'physiotherapy', 'maternity_care',
       'obstetrics', 'health_expenditure_under_deductible'],
      dtype='object')

Let's look again at the first couple of lines of our data set. The `head` method presents the first 5 line as default.



In [1]:
df_gemeente.head()

three-quebec-video-yankee

   sex             age MUNICIPALITY  number_citizens  hospital_care  \
0  NaN             NaN          NaN           298383    48661669.94
1    M   0 t/m  4 jaar  AA EN HUNZE              507      473370.77
2    M   0 t/m  4 jaar      AALBURG              428      340442.37
3    M   0 t/m  4 jaar     AALSMEER              876      932841.57
4    M   0 t/m  4 jaar       AALTEN              667      515879.78

   pharmaceuticals  mental_care  GP_capitation  GP_fee_for_service  \
0       9219422.33   7475481.90     4092492.35          1388439.07
1         43305.49     14056.81       26239.56            12829.29
2         23395.86      5555.83       21673.15             9417.98
3         54597.52     34355.66       43971.27            22413.37
4         43516.71      2404.19       33749.10            18472.64

     GP_other  dental care  physiotherapy  maternity_care  obstetrics  \
0  3041904.41   1249229.61      418648.30       1286545.5  1072906.37
1    36034.6

We are not interested in the first line, so we drop it. Indeed, our data set now starts with the first municipality 'AA EN HUNZE'.



In [1]:
df_gemeente.drop(df_gemeente.index[[0]], inplace=True)
df_gemeente.head()

seven-mango-cat-angel

  sex             age   MUNICIPALITY  number_citizens  hospital_care  \
1   M   0 t/m  4 jaar    AA EN HUNZE              507      473370.77
2   M   0 t/m  4 jaar        AALBURG              428      340442.37
3   M   0 t/m  4 jaar       AALSMEER              876      932841.57
4   M   0 t/m  4 jaar         AALTEN              667      515879.78
5   M   0 t/m  4 jaar  ACHTKARSPELEN              810      766456.80

   pharmaceuticals  mental_care  GP_capitation  GP_fee_for_service  GP_other  \
1         43305.49     14056.81       26239.56            12829.29  36034.65
2         23395.86      5555.83       21673.15             9417.98  20159.19
3         54597.52     34355.66       43971.27            22413.37  61629.32
4         43516.71      2404.19       33749.10            18472.64  46720.61
5         88084.16      6681.81       41477.79            19157.34  53633.01

   dental care  physiotherapy  maternity_care  obstetrics  \
1      9311.14       15968.80   

Now let's consider data types.



In [1]:
df_gemeente.dtypes

west-ohio-sodium-sodium

sex                                     object
age                                     object
MUNICIPALITY                            object
number_citizens                          int64
hospital_care                          float64
pharmaceuticals                        float64
mental_care                            float64
GP_capitation                          float64
GP_fee_for_service                     float64
GP_other                               float64
dental care                            float64
physiotherapy                          float64
maternity_care                         float64
obstetrics                             float64
health_expenditure_under_deductible    float64
dtype: object

The first three variables are seen as "object", that is, strings. This is fine for `MUNICIPALITY` but is not quite right for `sex` and `age` as these are categories. So let's relabel their types.



In [1]:
df_gemeente['sex'] = df_gemeente['sex'].astype('category')
df_gemeente['age'] = df_gemeente['age'].astype('category')
df_gemeente.info()

crazy-leopard-yankee-failed

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14808 entries, 1 to 14808
Data columns (total 15 columns):
sex                                    14808 non-null category
age                                    14808 non-null category
MUNICIPALITY                           14808 non-null object
number_citizens                        14808 non-null int64
hospital_care                          14808 non-null float64
pharmaceuticals                        14808 non-null float64
mental_care                            14808 non-null float64
GP_capitation                          14808 non-null float64
GP_fee_for_service                     14808 non-null float64
GP_other                               14808 non-null float64
dental care                            14808 non-null float64
physiotherapy                          14808 non-null float64
maternity_care                         14808 non-null float64
obstetrics                             14808 non-null flo

Now that we have the data ready, we are going to plot health care expenditures on the map of the Netherlands.




## geographical figures



We have map data that links the name of a municipality to coordinates
on the map. In this map data, the names of municipalities are
capitalized under standard Dutch capitalization like "Aa en Hunze". In
our Vektis data, the names of municipalities are written in
capitals. There are a number of ways to resolve this. To illustrate
the `merge` command, we use a file with two columns: 1. the names of
municipalities all capitalized and 2. normal capitalization. We drop
the rows where there is no value for municipality (if such rows
exist). We merge our data `df_gemeente` with the dataframe `Gemeentes`. We use a "left-merge", so all rows in the first dataframe `df_gemeente` are kept.



In [1]:
Gemeentes = pd.read_excel('Gemeentes.xlsx')
df_gemeente = df_gemeente.dropna(subset=['MUNICIPALITY'])
df_gem_merged = pd.merge(df_gemeente,Gemeentes,on=['MUNICIPALITY'],how='left')
df_gem_merged.head()

ack-louisiana-california-november

  sex             age   MUNICIPALITY  number_citizens  hospital_care  \
0   M   0 t/m  4 jaar    AA EN HUNZE              507      473370.77
1   M   0 t/m  4 jaar        AALBURG              428      340442.37
2   M   0 t/m  4 jaar       AALSMEER              876      932841.57
3   M   0 t/m  4 jaar         AALTEN              667      515879.78
4   M   0 t/m  4 jaar  ACHTKARSPELEN              810      766456.80

   pharmaceuticals  mental_care  GP_capitation  GP_fee_for_service  GP_other  \
0         43305.49     14056.81       26239.56            12829.29  36034.65
1         23395.86      5555.83       21673.15             9417.98  20159.19
2         54597.52     34355.66       43971.27            22413.37  61629.32
3         43516.71      2404.19       33749.10            18472.64  46720.61
4         88084.16      6681.81       41477.79            19157.34  53633.01

   dental care  physiotherapy  maternity_care  obstetrics  \
0      9311.14      

We are going to plot expenditure under the deductible per head for each municipality. We need a couple of steps in order to do this:

1.  we add &#x2013;for each municipality&#x2013; the expenditures under the deductible across age groups;
2.  we add &#x2013;for each municipality&#x2013; the number of people across age groups;
3.  we divide &#x2013;for each municipality&#x2013; the expenditures by the number of people.

With pandas this is straightforward to do using \`groupby\`. We do the \`groupby\` on the municipality. For each municipality there are different age groups and we need to aggregate over these age groups. We specify the variables that we want to know at the municipality level. In this case \`hospital\_care\` and \`number\_citizens\`. Finally, we specify how the function with which to aggregate. Here we use the 'built-in' function \`sum()\`. Other functions we can use include \`mean\`, \`min\`, \`max\` etc. You can also specify your own function and apply this using \`agg()\`.

Then hospital care per head can be defined as the total expenditure per municipality divided by the total number of citizens per municipality.

We can do other transformations of the data, such as the \`cost\_index\` below which is normalized to lie between 0 and 100.



In [1]:
results = df_gem_merged.groupby('Municipality')[['health_expenditure_under_deductible','number_citizens']].sum()
results['expenditure_per_head'] = results['health_expenditure_under_deductible']/results['number_citizens']
results.head()

thirteen-november-angel-delaware

                  health_expenditure_under_deductible  number_citizens  \
Municipality
's-Gravenhage                            9.115807e+08           509206
's-Hertogenbosch                         2.697598e+08           151934
Aa en Hunze                              4.696741e+07            25415
Aalburg                                  2.189368e+07            12871
Aalsmeer                                 4.864200e+07            31365

                  expenditure_per_head
Municipality
's-Gravenhage              1790.200148
's-Hertogenbosch           1775.506394
Aa en Hunze                1848.019292
Aalburg                    1701.008568
Aalsmeer                   1550.837018

In [1]:
# Create a dataframe
p_results = pd.DataFrame(dict(
    municipality = results.index,
    expenditure = results['expenditure_per_head']
    ))

p_results.head()

bravo-pip-march-batman

                  expenditure      municipality
Municipality
's-Gravenhage     1790.200148     's-Gravenhage
's-Hertogenbosch  1775.506394  's-Hertogenbosch
Aa en Hunze       1848.019292       Aa en Hunze
Aalburg           1701.008568           Aalburg
Aalsmeer          1550.837018          Aalsmeer

In [1]:
from IPython.display import display, IFrame
import folium

geo_path = r'Gemeentegrenzen_2016_zonder_water_simplified_wgs84.geojson'


ref_map = folium.Map(
    location=[52.139177, 5.327108], # This will center the view on the world map where the Netherlands is located
    tiles='Mapbox Bright',          # This creates a base map and in this case its the Mapbox Bright basemap
    zoom_start=8)                   # This will zoom in on the center of view to get the Netherlands in full frame

ref_map.choropleth(
    # This is the path to the geojson file that contains all the municipality shapes and locations
    geo_path=geo_path,
    # We will use the percentage dataframe for the choropleth mapping
    data=p_results,
    # Municipality will be used for the mapping key and p_valid for its value
    columns=['municipality', 'expenditure'],
    # Use GM_NAAM (short for municipality name) as keys for colormapping
    key_on='feature.properties.GM_NAAM',
    # We are going to use a color map from yellow to green
    fill_color='YlGn',
    # This gives municipality shapes some opacity so that we can still see the background
    fill_opacity=0.7,
    # This gives the lines around the municipality shapes some opacity so that they don't stand out too much
    line_opacity=0.2,
    # The legend
    legend_name='health care expenditure per head')

ref_map.save('health_expenditure.html')              # This will save the map in a HTML format

display(IFrame('health_expenditure.html', 800,800))  # This displays the map in an Iframe

muppet-lamp-william-minnesota



# Health care expenditure and age



The municipality data set above does not give the health care expenditure per age; only per age group (like 0-4 year olds). So we load another data set that does feature health care expenditure per age.




## read in the data



Again, we use data from [Vektis](http://www.vektis.nl/index.php/vektis-open-data). We import the data, which is a 'csv' file with ";" as separator between columns. We also import some libraries.

Then we look at the columns (variables) in the data.



In [1]:
import numpy as np
import matplotlib as plt
df_postal_code = pd.read_csv('Vektis_Open_Databestand_Zorgverzekeringswet_2014_-_postcode3.csv', sep = ';')
df_postal_code.dtypes

This looks very much like the data set above, so we want to do the same steps to get the data into the shape we want. In fact, if you go to the website [Vektis](http://www.vektis.nl/index.php/vektis-open-data) there are similar data sets for other years. Copy and paste the steps above and then apply these steps to the new data sets is asking for trouble:

-   you are likely to make mistakes with copy/paste
-   if you figure out that you want to change one of your commands, you have to change all the pasted versions as well

One solution in python is to define a function that does all these steps for you and apply this function to all the data sets that you want to work with.



In [1]:
def get_data_into_shape(df):
    df['health_expenditure_under_deductible'] = df[cost_categories_under_deductible].sum(axis=1)
    df = df.rename_axis({
        'GESLACHT':'sex',
        'LEEFTIJDSKLASSE':'age',
        'GEMEENTENAAM':'MUNICIPALITY',
        'AANTAL_BSN':'number_citizens',
        'KOSTEN_MEDISCH_SPECIALISTISCHE_ZORG':'hospital_care',
        'KOSTEN_FARMACIE':'pharmaceuticals',
        'KOSTEN_TWEEDELIJNS_GGZ':'mental_care',
        'KOSTEN_HUISARTS_INSCHRIJFTARIEF':'GP_capitation',
        'KOSTEN_HUISARTS_CONSULT':'GP_fee_for_service',
        'KOSTEN_HUISARTS_OVERIG':'GP_other',
        'KOSTEN_MONDZORG':'dental care',
        'KOSTEN_PARAMEDISCHE_ZORG_FYSIOTHERAPIE':'physiotherapy',
        'KOSTEN_KRAAMZORG':'maternity_care',
        'KOSTEN_VERLOSKUNDIGE_ZORG':'obstetrics'
    }, axis='columns')
    df.drop(['AANTAL_VERZEKERDEJAREN',
             'KOSTEN_HULPMIDDELEN',
             'KOSTEN_PARAMEDISCHE_ZORG_OVERIG',
             'KOSTEN_ZIEKENVERVOER_ZITTEND',
             'KOSTEN_ZIEKENVERVOER_LIGGEND',
             'KOSTEN_GRENSOVERSCHRIJDENDE_ZORG',
             'KOSTEN_GERIATRISCHE_REVALIDATIEZORG',
             'KOSTEN_OVERIG',
             'KOSTEN_GENERALISTISCHE_BASIS_GGZ',
             'KOSTEN_EERSTELIJNS_ONDERSTEUNING'],inplace=True,axis=1)
    df.drop(df.index[[0]], inplace=True)
    df['sex'] = df['sex'].astype('category')
    df['age'] = df['age'].astype('category')
    return df

In [1]:
df_postal_code = get_data_into_shape(df_postal_code)

In [1]:
df_postal_code.head()

delaware-beryllium-north-three

  sex age  POSTCODE_3  number_citizens  hospital_care  pharmaceuticals  \
1   M   0         0.0              366     1372209.26         31191.20
2   M   0       101.0              590     1682944.17         25898.73
3   M   0       102.0              295     1553933.53         29514.18
4   M   0       103.0              288      827427.31         19263.79
5   M   0       105.0              998     2965316.12         61610.42

   mental_care  GP_capitation  GP_fee_for_service  GP_other  dental care  \
1       285.98        5548.60             5540.05  11525.93       681.02
2     20774.91        9816.63            10130.12  20532.03         0.00
3      7970.01        5317.49             6576.70  17426.30        21.29
4       941.40        5014.97             5708.41  14168.90         0.00
5      4780.48       16842.06            19676.01  43794.06       166.98

   physiotherapy  maternity_care  obstetrics  \
1       12150.91             0.0         0.0
2  

The first three columns are 'sex', 'age' and 'postal code' (3 digit). These 3 variables combined determine a unique observation. We think of these observations as if they are from an individual (but an observation is an average, like the average over 18 year old males in postal code 102).

Note that the first postal code is '000' which python thinks of as '0.0'. The
[data description](http://www.vektis.nl/images/open_data/Bijsluiter_bij_de_Vektis_Open_Databestanden_Zorgverzekeringswet_2011_-_2014.pdf) explains that postal code '000' is used to aggregate people who
live in a postal code with so few people that the privacy of their data is no
longer guaranteed. As we want to think of \`sex\`, \`age\` and \`postal code\` as an observation, we drop the first row (labelled as \`0\`) of the dataframe.



In [1]:
df_postal_code.drop(df_postal_code.index[[0]], inplace=True)
df_postal_code.head()

one-queen-arizona-venus

  sex age  POSTCODE_3  number_citizens  hospital_care  pharmaceuticals  \
2   M   0       101.0              590     1682944.17         25898.73
3   M   0       102.0              295     1553933.53         29514.18
4   M   0       103.0              288      827427.31         19263.79
5   M   0       105.0              998     2965316.12         61610.42
6   M   0       106.0             1056     3716382.22         87140.60

   mental_care  GP_capitation  GP_fee_for_service  GP_other  dental care  \
2     20774.91        9816.63            10130.12  20532.03         0.00
3      7970.01        5317.49             6576.70  17426.30        21.29
4       941.40        5014.97             5708.41  14168.90         0.00
5      4780.48       16842.06            19676.01  43794.06       166.98
6     25006.18       19517.84            24045.35  65572.64       114.05

   physiotherapy  maternity_care  obstetrics  \
2       17777.00             0.0         0.0
3       20

The end of the dataframe is given by the following.



In [1]:
df_postal_code.tail(10)

finch-oven-thirteen-nine

       sex  age  POSTCODE_3  number_citizens  hospital_care  pharmaceuticals  \
136463   V  90+       988.0               10       19698.83          4011.31
136464   V  90+       990.0              151      257046.54         99187.66
136465   V  90+       991.0               51       95990.43         52682.34
136466   V  90+       993.0              170      278000.11        124809.41
136467   V  90+       994.0               38       28454.41         36590.90
136468   V  90+       995.0               88      200183.72         64315.53
136469   V  90+       996.0               44       46723.13         39419.64
136470   V  90+       997.0               38       98954.45         34308.68
136471   V  90+       998.0              116      168802.54        116907.93
136472   V  90+       999.0               38      109842.07         40607.06

        mental_care  GP_capitation  GP_fee_for_service  GP_other  dental care  \
136463         0.00         894.25        

As we saw above, the datatype of \`age\` was \`object\`, although we would expect \`integer\`. Now we see that there is this category \`90+\`, which is not an integer. We will drop this age category as it is quite special. Before we do this, let's count how many people we have in our dataset.



In [1]:
df_postal_code['number_citizens'].sum()

comet-sodium-sink-kansas

16885677

That is, almost 17 million people, which is about right.

Let's drop the '90+' category and turn \`age\` into an integer variable.



In [1]:
df_postal_code = df_postal_code[(df_postal_code['age'] != '90+')]
df_postal_code['age'] = df_postal_code['age'].astype(int)

nevada-nevada-alpha-lithium

nil:END:

Let's summarize the variables that are numeric (integer or float). For each of these variables we have 135,063 observations (that is, combinations of `sex`, `age` and `postal code`). The mean for number of citizens is 124. With $135,063*124$ we are close to 17 million again.

#+NAME: mexico-minnesota-carbon-single
#+BEGIN_SRC ipython
df_postal_code.describe()
#+END_SRC

 mexico-minnesota-carbon-single

                 age     POSTCODE_3  number_citizens  hospital_care  \
count  135063.000000  135063.000000    135063.000000   1.350630e+05
mean       43.753959     541.250002       123.950327   1.534887e+05
std        25.535582     258.016742       129.491359   1.965840e+05
min         0.000000       0.000000        10.000000  -2.300980e+04
25%        22.000000     318.000000        40.000000   3.055294e+04
50%        44.000000     539.000000        82.000000   8.262627e+04
75%        66.000000     763.000000       161.000000   2.006986e+05
max     

    df_postal_code.describe()

                 age     POSTCODE\_3  number\_citizens  hospital\_care  \\
count  135063.000000  135063.000000    135063.000000   1.350630e+05
mean       43.753959     541.250002       123.950327   1.534887e+05
std        25.535582     258.016742       129.491359   1.965840e+05
min         0.000000       0.000000        10.000000  -2.300980e+04
25%        22.000000     318.000000        40.000000   3.055294e+04
50%        44.000000     539.000000        82.000000   8.262627e+04
75%        66.000000     763.000000       161.000000   2.006986e+05
max        89.000000     999.000000      2228.000000   5.263426e+06

       pharmaceuticals    mental\_care  GP\_capitation  GP\_fee\_for\_service  \\
count     1.350630e+05  135063.000000  135063.000000       135063.000000
mean      3.146225e+04   23296.150212    7693.165768         4528.021995
std       4.483099e+04   45078.454602    7676.781993         5060.276133
min      -2.857890e+03  -29164.050000       0.000000            0.000000
25%       5.034130e+03     299.260000    2585.970000         1351.340000
50%       1.547225e+04    5702.020000    5244.990000         2900.080000
75%       4.037718e+04   24546.195000   10140.510000         5899.600000
max       1.546412e+06  885045.050000  155453.330000       194903.830000

            GP\_other    dental care  physiotherapy  maternity\_care  \\
count  135063.000000  135063.000000  135063.000000   135063.000000
mean     5938.297319    5438.213219    3287.127334     2201.230080
std      6329.471736   10190.600213    4659.495994    10426.252898
min         0.000000    -458.190000    -106.800000    -1898.420000
25%      1934.255000     150.075000     143.745000        0.000000
50%      4002.870000    1746.990000    1617.300000        0.000000
75%      7691.255000    6293.135000    4463.975000        0.000000
max    276119.620000  254585.130000  106169.130000   399960.460000

          obstetrics  health\_expenditure\_under\_deductible
count  135063.000000                         1.350630e+05
mean     1612.545851                         2.192913e+05
std      7853.598487                         2.719250e+05
min         0.000000                         0.000000e+00
25%         0.000000                         4.791477e+04
50%         0.000000                         1.234374e+05
75%         0.000000                         2.872767e+05
max    321751.460000                         9.012553e+06



In [1]:
df_postal_code.describe()

mexico-minnesota-carbon-single

                 age     POSTCODE_3  number_citizens  hospital_care  \
count  135063.000000  135063.000000    135063.000000   1.350630e+05
mean       43.753959     541.250002       123.950327   1.534887e+05
std        25.535582     258.016742       129.491359   1.965840e+05
min         0.000000       0.000000        10.000000  -2.300980e+04
25%        22.000000     318.000000        40.000000   3.055294e+04
50%        44.000000     539.000000        82.000000   8.262627e+04
75%        66.000000     763.000000       161.000000   2.006986e+05
max        89.000000     999.000000      2228.000000   5.263426e+06

       pharmaceuticals    mental_care  GP_capitation  GP_fee_for_service  \
count     1.350630e+05  135063.000000  135063.000000       135063.000000
mean      3.146225e+04   23296.150212    7693.165768         4528.021995
std       4.483099e+04   45078.454602    7676.781993         5060.276133
min      -2.857890e+03  -29164.050000       0.000000     

In [1]:
df_postal_code.info()

Now let's define the costs per head. For each observation, we divide the total health care costs (under the deductible) for a combination of \`sex\`, \`age\` and \`postal code\` by the number of people in this combination of \`sex\`, \`age\` and \`postal code\`. This gives the health costs per head.



In [1]:
df_postal_code['health_costs_per_head'] = df_postal_code['health_expenditure_under_deductible']/df_postal_code['number_citizens']

So for, say, 18 year old males, we have a distribution of costs per head over the different \`postal codes\`. For each combination of age and sex, we can look at the average expenditure. With `pandas` this is easy to do. We use `groupby`, specify the dimensions over which we want to group, the variable we are interested in and give the function to aggregate (mean, in this case).



In [1]:
costs_per_sex_age = df_postal_code.groupby(['sex','age'])['health_costs_per_head'].mean()

Then we can plot this distribution of health care expenditure per head with age for males and females.



In [1]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')
fig = plt.figure()
ax = costs_per_sex_age['M'].plot()
ax = costs_per_sex_age['V'].plot()
ax.set_xlabel('age')
ax.set_ylabel('costs per head')
ax.set_title('average costs per age and sex')
ax.legend(['male','female'])
fig.savefig("males.png")

![img](./males.png)

Suppose you are interested in the effect of the deductible on health care expenditure. Why would the following graph help for this?



In [1]:
plt.style.use('seaborn')
plt.clf()
age_range = [14,15,16,17,19,20,21,22]

plt.plot(age_range,costs_per_sex_age['M'][age_range], marker='.', label = 'male')
plt.plot(age_range,costs_per_sex_age['V'][age_range], marker='.', label = 'female')
plt.xlabel('age')
plt.ylabel('health care costs')
plt.legend()
fig.savefig('fig14to22.png')

![img](./fig14to22.png)

Instead of `matplotlib` to plot, we can also use `plotly`. With `plotly` you can make interactive graphs.



In [1]:
def ecdf(data):
    x = np.sort(data)
    y = np.arange(1.0, len(x)+1.0) / len(x)
    return x, y

In [1]:
df_postal_code.health_costs_per_head[(df_postal_code['age'] == 17)]

In [1]:
x_16, y_16 = ecdf(df_postal_code.health_costs_per_head[(df_postal_code['age'] == 16)])
x_17, y_17 = ecdf(df_postal_code.health_costs_per_head[(df_postal_code['age'] == 17)])
x_19, y_19 = ecdf(df_postal_code.health_costs_per_head[(df_postal_code['age'] == 19)])
x_20, y_20 = ecdf(df_postal_code.health_costs_per_head[(df_postal_code['age'] == 20)])

In [1]:
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

In [1]:
age16 = Scatter(
    x=x_16,
    y=y_16,
    mode='markers+lines',
    name = 'age 16'
)
age17 = Scatter(
    x=x_17,
    y=y_17,
    mode='markers+lines',
    name = 'age 17'
)
age19 = Scatter(
    x=x_19,
    y=y_19,
    mode='markers+lines',
    name = 'age 19'
)
age20 = Scatter(
    x=x_20,
    y=y_20,
    mode='markers+lines',
    name = 'age 20'
)






layout = Layout(
    title='Health care expend. distribution functions',
    xaxis=XAxis(
        range=[0,3000],
        title='expenditure per head',
        titlefont=Font(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=YAxis(
        title='cum. distribution function',
        titlefont=Font(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

data = Data([age16,age17,age19,age20])
fig = Figure(data=data, layout=layout)
py.plot(fig, filename='Distribution functions of health care expenditure per head')
tls.embed("https://plot.ly/~janboone/301")

artist-mockingbird-florida-hot

<plotly.tools.PlotlyDisplay object>